In [1]:
# for e in driver.find_elements(By.CLASS_NAME, "SaleSectionForCustomCSS"):
#     if "filters" in (e.text.lower()):
#         y = e
#         print(e)

Setting the Enviroment

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By 
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException

import pandas as pd
import time

# CONSTANTS
SERVICE = Service(ChromeDriverManager().install())
DRIVER = webdriver.Chrome(service=SERVICE)
URL = "https://store.steampowered.com/specials"
GAMES_CLASS_NAME = "ImpressionTrackedElement"

# Navigate to the desired URL
DRIVER.get(URL) 

BODY = DRIVER.find_element(By.TAG_NAME, "body")

In [3]:
REVIEWS = [
    "Overwhelmingly Positive"
    , "Very Positive"
    , "Mostly Positive"
    , "Mixed"
    , "Mostly Negative"
    , "Very Negative"
    , "Overwhelmingly Negative"
]

In [4]:
TABS_CONTAINER = None

while not TABS_CONTAINER:
    BODY.send_keys(Keys.PAGE_DOWN)
    TABS_CONTAINER = DRIVER.find_elements(By.XPATH, "//div[contains(@class, 'Focusable') and @tabindex='0' and text()]")

In [5]:
time.sleep(2)

MAIN_CONTAINER = TABS_CONTAINER[0]

while "$" not in MAIN_CONTAINER.text:
    MAIN_CONTAINER = MAIN_CONTAINER.find_element(By.XPATH, "..")

GAMES_CONTAINER = MAIN_CONTAINER.find_elements(By.CLASS_NAME, GAMES_CLASS_NAME)

In [6]:
while True:
    try:
        DRIVER.execute_script("arguments[0].click();", TABS_CONTAINER[0])
        break
    except ElementClickInterceptedException:
        print("Trying")
        TABS_CONTAINER[0].send_keys(Keys.PAGE_UP)
        time.sleep(1)  

In [ ]:
for tab in TABS_CONTAINER:
    
    while True:
        try:
            DRIVER.execute_script("arguments[0].click();", tab)
            break
        except ElementClickInterceptedException:
            print("Trying")
            tab.send_keys(Keys.PAGE_UP)
            time.sleep(1)  

    DATA_DICT = {
    "Name" : []
    , "Reviews" : []
    , "Reviews Number" : []
    , "Discount" : []
    , "Original Price" : []
    , "Discounted Price" : []
}

    for game_container in GAMES_CONTAINER:

        game_container = game_container.text.strip()
        game_container = game_container.split("\n")
    
        DATA_DICT["Name"].append(game_container[0])
        
        reviews = [game_string for game_string in game_container if game_string in REVIEWS]
        reviews = str(reviews).replace('[', '').replace(']', '')
        DATA_DICT["Reviews"].append(reviews)
        
        reviews_number = [game_string for game_string in game_container if "User Reviews" in game_string]
        reviews_number = str(reviews_number).replace('[', '').replace(']', '')
        DATA_DICT["Reviews Number"].append(reviews_number)

        discount = [game_string for game_string in game_container if game_string.endswith("%")]  
        discount = str(discount).replace('[', '').replace(']', '') 
        if not discount:
            discount = None    
        DATA_DICT["Discount"].append(discount)

        price = [game_string for game_string in game_container if game_string.startswith("$")]

        if price:
            DATA_DICT["Original Price"].append(price[0])
            if len(price)>1:
                DATA_DICT["Discounted Price"].append(price[1])           
        else:
            price = "Coming Soon"
            DATA_DICT["Original Price"].append(price)
            DATA_DICT["Discounted Price"].append(None)

    print(pd.DataFrame.from_dict(DATA_DICT))

In [9]:
for key,value in DATA_DICT.items():
    print(key)
    print(len(value))
    print(value)

Name
12
['Grand Theft Auto V', 'FINAL FANTASY VII REBIRTH', 'Black Desert', 'Sengoku Dynasty', 'Elite Dangerous', 'Beyond Citadel', 'BioShock Infinite', 'OUTBRK', 'LEGO® Bricktales', 'Automation - The Car Company Tycoon Game', 'Tainted Grail: The Fall of Avalon', 'shapez 2']
Reviews
12
["'Very Positive'", '', '', "'Mostly Positive'", "'Mostly Positive'", "'Very Positive'", "'Very Positive'", "'Mostly Positive'", "'Very Positive'", "'Very Positive'", "'Very Positive'", "'Overwhelmingly Positive'"]
Reviews Number
12
["'1,764,976 User Reviews'", '', '', "'4,332 User Reviews'", "'73,761 User Reviews'", "'118 User Reviews'", "'103,942 User Reviews'", "'1,291 User Reviews'", "'932 User Reviews'", "'12,317 User Reviews'", "'3,082 User Reviews'", "'8,883 User Reviews'"]
Discount
12
[None, "'-30%'", "'-100%'", "'-25%'", "'-75%'", "'-10%'", "'-75%'", "'-25%'", "'-75%'", "'-25%'", "'-30%'", "'-25%'"]
Original Price
12
['$14.80', '$69.99', '$9.99', '$29.99', '$19.99', '$14.99', '$29.99', '$34.99',

# Less Garbage

In [ ]:
element = driver.find_element(By.TAG_NAME, "body")
ATTEMPT_NUMBER = 0

while True:
    try: 
        top_sellers = driver.find_element(By.XPATH, "//*[text()='Top Sellers']")
        break
    except NoSuchElementException:
        ATTEMPT_NUMBER += 1
        print(f"Attempt #{ATTEMPT_NUMBER} to Find the Element...") 
        element.send_keys(Keys.PAGE_DOWN)

#is enabled
#is_displayed

In [ ]:
GAMES_LISTS = ["Top Sellers", "New & Trending", "Top Rated", "Discounted", "Upcoming"]
GAMES_CLASS_NAME = "ImpressionTrackedElement"



games_lists_elements = [driver.find_element(By.XPATH, f"//*[text()='{LIST}']") for LIST in GAMES_LISTS]

In [ ]:
for games_list_element in games_lists_elements:
    games_list_element.click()

    DATA_DICT = {
    "Name" : []
    , "Reviews" : []
    , "Reviews Number" : []
    , "Discount" : []
    , "Original Price" : []
    , "Discounted Price" : []
}

    while "$" not in games_list_element.text:
        games_list_element = games_list_element.find_element(By.XPATH, "..")

    for game in games_list_element.find_elements(By.CLASS_NAME, GAMES_CLASS_NAME):

        game = game.text.strip().split("\n")
        game = [g_str.strip() for g_str in game]

        DATA_DICT["Name"].append(game[0])
        DATA_DICT["Reviews"].append(game[3]) 
        DATA_DICT["Reviews Number"].append(game[4])
        DATA_DICT["Discount"].append(game[6])
        DATA_DICT["Original Price"].append(game[7]) 
        DATA_DICT["Discounted Price"].append(game[8])

    print(pd.DataFrame.from_dict(DATA_DICT))

# Garbage

In [ ]:
#Loading Dependencies

import pandas as pd
import requests as r
from bs4 import BeautifulSoup as bs

In [ ]:
#Creating the Soup

base_url = "https://store.steampowered.com/specials"
html_content = r.get(base_url).content
soup = bs(html_content, "lxml")

#Exporting the Soup for Readability
with open("steam_store_specials2.html", "wb") as file:
    file.write(html_content)

Nested Tree

In [ ]:
# div tab_content_ctn sub
#     h2 tab_content_title
#     div tab_content_section_ctn
#         a class="tab_item"
#             div class="tab_item_name"
#             div class="discount_pct"
#             div class="discount_original_price"
#             div class="discount_final_price"

In [ ]:
soup.find_all("div", class_="Eh-ow8RpArn1HQyPR8RFY Panel Focusable")

Creating the Container

In [ ]:
container = soup.find('div', class_ = 'tab_content_ctn sub')
type(container)

Extracting the Tabs Names

In [ ]:
tabs_names = [c.text for c in container.find_all('h2')]
tabs_names

df = pd.DataFrame(columns=tabs_names)
df

Extracting the Tabs Content

In [ ]:
tabs_content = container.find_all('div', class_ = 'tab_content_section_ctn')
type(tabs_content)

Extracting the Games Details from Each Tab

In [ ]:
games_details = [c.find_all('a', class_="tab_item") for c in tabs_content]
type(games_details)

In [ ]:
#Extracting the Name of Each Game from Each tab

game_name = [[l.find('div', class_="tab_item_name").text for l in g] for g in games_details]
game_name

In [ ]:
#Extracting the Original Price of Each Game from Each tab
game_original_price = [[float((l.find('div', class_="discount_original_price").text).strip('$')) for l in g] for g in games_details]
game_original_price

In [ ]:
#Extracting the Discount % of Each Game from Each tab
game_discount = [[int((l.find('div', class_="discount_pct").text).strip('-%')) for l in g] for g in games_details]
game_discount

In [ ]:
#Extracting the Final Price of Each Game from Each tab
game_final_price = [[float((l.find('div', class_="discount_final_price").text).strip('$')) for l in g] for g in games_details]
game_final_price

In [ ]:
idx = 0
pd_data = [[],[],[],[]]
columns_list = ['Name', 'Original Price', 'Discount', "Final Price"]

for i in range(4):
    pd_data[idx] = pd.DataFrame([game_name[idx], game_original_price[idx], game_discount[idx], game_final_price[idx]], index=columns_list).T
    idx += 1

In [ ]:
pd_data[0]

In [ ]:
pd_data[1]

In [ ]:
pd_data[2]

In [ ]:
pd_data[3]

Exporting the Data

In [ ]:
new_and_trending = pd.DataFrame()
top_sellers = pd.DataFrame()
being_played = pd.DataFrame()
pre_purchase = pd.DataFrame()

new_and_trending['Name'] = game_name[0]
new_and_trending['Original Price'] = game_original_price[0]
new_and_trending['Discount %'] = game_discount[0]
new_and_trending['Final Price'] = game_final_price[0]

top_sellers['Name'] = game_name[1]
top_sellers['Original Price'] = game_original_price[1]
top_sellers['Discount %'] = game_discount[1]
top_sellers['Final Price'] = game_final_price[1]

being_played['Name'] = game_name[2]
being_played['Original Price'] = game_original_price[2]
being_played['Discount %'] = game_discount[2]
being_played['Final Price'] = game_final_price[2]

pre_purchase['Name'] = game_name[3]
pre_purchase['Original Price'] = game_original_price[3]
pre_purchase['Discount %'] = game_discount[3]
pre_purchase['Final Price'] = game_final_price[3]

In [ ]:
new_and_trending

In [ ]:
top_sellers

In [ ]:
being_played

In [ ]:
pre_purchase